# Siren Exploration

This is a colab to explore properties of the Siren MLP, proposed in our work [Implicit Neural Activations with Periodic Activation Functions](https://vsitzmann.github.io/siren).


We will first implement a streamlined version of Siren for fast experimentation. This lacks the code to easily do baseline comparisons - please refer to the main code for that - but will greatly simplify the code!

**Make sure that you have enabled the GPU under Edit -> Notebook Settings!**

We will then reproduce the following results from the paper:
* [Fitting an image](#section_1)
* [Fitting an audio signal](#section_2)
* [Solving Poisson's equation](#section_3)
* [Initialization scheme & distribution of activations](#activations)
* [Distribution of activations is shift-invariant](#shift_invariance)

We will also explore Siren's [behavior outside of the training range](#out_of_range).

Let's go! First, some imports, and a function to quickly generate coordinate grids.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt

import time

def get_mgrid(sidelen, dim=2):
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''
    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])
    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)
    mgrid = mgrid.reshape(-1, dim)
    return mgrid

Now, we code up the sine layer, which will be the basic building block of SIREN. This is a much more concise implementation than the one in the main code, as here, we aren't concerned with the baseline comparisons.

In [2]:
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.

    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a
    # hyperparameter.

    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)

    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first

        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features,
                                             1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0,
                                             np.sqrt(6 / self.in_features) / self.omega_0)

    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))

    def forward_with_intermediate(self, input):
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate


class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False,
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()

        self.net = []
        self.net.append(SineLayer(in_features, hidden_features,
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features,
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)

            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0,
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)

            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features,
                                      is_first=False, omega_0=hidden_omega_0))

        self.net = nn.Sequential(*self.net)

    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords

    def forward_with_activations(self, coords, retain_grad=False):
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations['input'] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)

                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()

                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else:
                x = layer(x)

                if retain_grad:
                    x.retain_grad()

            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

And finally, differential operators that allow us to leverage autograd to compute gradients, the laplacian, etc.

In [3]:
def laplace(y, x):
    grad = gradient(y, x)
    return divergence(grad, x)


def divergence(y, x):
    div = 0.
    for i in range(y.shape[-1]):
        div += torch.autograd.grad(y[..., i], x, torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
    return div


def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

# Experiments

For the image fitting and poisson experiments, we'll use the classic cameraman image.

In [4]:
def get_cameraman_tensor(sidelength):
    img = Image.fromarray(skimage.data.camera())
    transform = Compose([
        Resize(sidelength),
        ToTensor(),
        Normalize(torch.Tensor([0.5]), torch.Tensor([0.5]))
    ])
    img = transform(img)
    return img

<a id='section_1'></a>
## Fitting an image

First, let's simply fit that image!

We seek to parameterize a greyscale image $f(x)$ with pixel coordinates $x$ with a SIREN $\Phi(x)$.

That is we seek the function $\Phi$ such that:
$\mathcal{L}=\int_{\Omega} \lVert \Phi(\mathbf{x}) - f(\mathbf{x}) \rVert\mathrm{d}\mathbf{x}$
 is minimized, in which $\Omega$ is the domain of the image.

We write a little datast that does nothing except calculating per-pixel coordinates:

In [5]:
class ImageFitting(Dataset):
    def __init__(self, sidelength):
        super().__init__()
        img = get_cameraman_tensor(sidelength)
        self.pixels = img.permute(1, 2, 0).view(-1, 1)
        self.coords = get_mgrid(sidelength, 2)

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        if idx > 0: raise IndexError

        return self.coords, self.pixels

Let's instantiate the dataset and our Siren. As pixel coordinates are 2D, the siren has 2 input features, and since the image is grayscale, it has one output channel.

In [6]:
cameraman = ImageFitting(256)
dataloader = DataLoader(cameraman, batch_size=1, pin_memory=True, num_workers=0)

img_siren = Siren(in_features=2, out_features=1, hidden_features=256,
                  hidden_layers=3, outermost_linear=True)
img_siren.cuda()

C:\Users\stevenT\PycharmProjects\SIREN\.venv\Lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Siren(
  (net): Sequential(
    (0): SineLayer(
      (linear): Linear(in_features=2, out_features=256, bias=True)
    )
    (1): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (2): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

We now fit Siren in a simple training loop. Within only hundreds of iterations, the image and its gradients are approximated well.

In [7]:
# import wandb
# wandb.login()
#

#
# epochs = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.
# steps_til_summary = 10
# learning_rate = 1e-4
#
# run = wandb.init(
#     project="SIREN_tryout2",
#     config={
#         "epochs": epochs,
#         "learning rate": learning_rate,
#     },
# )
#
# optim = torch.optim.Adam(lr=learning_rate, params=img_siren.parameters())
#
# model_input, ground_truth = next(iter(dataloader))
# model_input, ground_truth = model_input.cuda(), ground_truth.cuda()
#
# for step in range(epochs):
#     model_output, coords = img_siren(model_input)
#     loss = ((model_output - ground_truth)**2).mean()
#
#     if not step % steps_til_summary:
#         print("Step %d, Total loss %0.6f" % (step, loss))
#         img_grad = gradient(model_output, coords)
#         img_laplacian = laplace(model_output, coords)
#
#         fig, axes = plt.subplots(1,3, figsize=(18,6))
#         axes[0].imshow(model_output.cpu().view(256,256).detach().numpy())
#         axes[1].imshow(img_grad.norm(dim=-1).cpu().view(256,256).detach().numpy())
#         axes[2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
#         plt.show()
#
#     optim.zero_grad()
#     loss.backward()
#     optim.step()
#
#     run.log({
#         "loss": loss,
#         "mse": mse(ground_truth, model_output),
#         "psnr": psnr(ground_truth, model_output),
#     })
# run.finish()

In [11]:
import wandb

def psnr(im1, im2, max_pixel_value=1):
    return 20 * torch.log10(max_pixel_value / mse(im1, im2))

def mse(im1, im2):
    return F.mse_loss(im1, im2)

wandb.login()

epochs = 500
learning_rate = 1e-4


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input.to(device), ground_truth.to(device)

for omega in [10, 20, 30, 40]:
    run = wandb.init(
        project="SIREN_omegas",
        name=f"SIREN_omegas_{omega}",
        config={
            "epochs": epochs,
            "learning rate": learning_rate,
            "omega": omega,
        },
    )

    model = Siren(in_features=2, out_features=1, hidden_features=256,
                  hidden_layers=3, outermost_linear=True,  hidden_omega_0=omega, first_omega_0=omega).to(device)

    optim = torch.optim.Adam(lr=learning_rate, params=model.parameters())

    for epoch in range(epochs):
        model_output, coords = model(model_input)

        loss = mse(ground_truth, model_output)

        optim.zero_grad()
        loss.backward()
        optim.step()

        run.log({
            "loss": loss,
            "mse": mse(ground_truth, model_output),
            "psnr": psnr(ground_truth, model_output),
        })

        if not epoch % 10:
            print(f"Epoch {epoch}: loss = {mse(ground_truth, model_output):.4f}, psnr = {psnr(ground_truth, model_output):.4f}")

    run.finish()



Epoch 0: loss = 0.3268, psnr = 9.7155
Epoch 10: loss = 0.0555, psnr = 25.1104
Epoch 20: loss = 0.0427, psnr = 27.3993
Epoch 30: loss = 0.0358, psnr = 28.9305
Epoch 40: loss = 0.0322, psnr = 29.8296
Epoch 50: loss = 0.0302, psnr = 30.4035
Epoch 60: loss = 0.0285, psnr = 30.9109
Epoch 70: loss = 0.0271, psnr = 31.3418
Epoch 80: loss = 0.0259, psnr = 31.7243
Epoch 90: loss = 0.0249, psnr = 32.0660
Epoch 100: loss = 0.0241, psnr = 32.3746
Epoch 110: loss = 0.0233, psnr = 32.6580
Epoch 120: loss = 0.0226, psnr = 32.9249
Epoch 130: loss = 0.0219, psnr = 33.1803
Epoch 140: loss = 0.0213, psnr = 33.4265
Epoch 150: loss = 0.0207, psnr = 33.6635
Epoch 160: loss = 0.0202, psnr = 33.8904
Epoch 170: loss = 0.0197, psnr = 34.1066
Epoch 180: loss = 0.0192, psnr = 34.3121
Epoch 190: loss = 0.0188, psnr = 34.5076
Epoch 200: loss = 0.0184, psnr = 34.6945
Epoch 210: loss = 0.0180, psnr = 34.8745
Epoch 220: loss = 0.0177, psnr = 35.0495
Epoch 230: loss = 0.0173, psnr = 35.2214
Epoch 240: loss = 0.0170, ps

loss,█▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,▁▂▃▃▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
loss,0.0105
mse,0.0105
psnr,39.57323


Epoch 0: loss = 0.3246, psnr = 9.7736
Epoch 10: loss = 0.0423, psnr = 27.4659
Epoch 20: loss = 0.0269, psnr = 31.4202
Epoch 30: loss = 0.0218, psnr = 33.2338
Epoch 40: loss = 0.0189, psnr = 34.4593
Epoch 50: loss = 0.0172, psnr = 35.2951
Epoch 60: loss = 0.0158, psnr = 36.0044
Epoch 70: loss = 0.0148, psnr = 36.5893
Epoch 80: loss = 0.0140, psnr = 37.1045
Epoch 90: loss = 0.0132, psnr = 37.5732
Epoch 100: loss = 0.0126, psnr = 38.0086
Epoch 110: loss = 0.0120, psnr = 38.4269
Epoch 120: loss = 0.0114, psnr = 38.8399
Epoch 130: loss = 0.0109, psnr = 39.2541
Epoch 140: loss = 0.0104, psnr = 39.6728
Epoch 150: loss = 0.0099, psnr = 40.0993
Epoch 160: loss = 0.0094, psnr = 40.5369
Epoch 170: loss = 0.0089, psnr = 40.9870
Epoch 180: loss = 0.0085, psnr = 41.4494
Epoch 190: loss = 0.0080, psnr = 41.9209
Epoch 200: loss = 0.0076, psnr = 42.3934
Epoch 210: loss = 0.0072, psnr = 42.8610
Epoch 220: loss = 0.0068, psnr = 43.3224
Epoch 230: loss = 0.0065, psnr = 43.7816
Epoch 240: loss = 0.0061, ps

loss,█▇▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▆▆▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,▁▁▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████
loss,0.0017
mse,0.0017
psnr,55.37947


Epoch 0: loss = 0.3226, psnr = 9.8258
Epoch 10: loss = 0.0445, psnr = 27.0250
Epoch 20: loss = 0.0215, psnr = 33.3659
Epoch 30: loss = 0.0162, psnr = 35.8017
Epoch 40: loss = 0.0135, psnr = 37.3672
Epoch 50: loss = 0.0120, psnr = 38.4302
Epoch 60: loss = 0.0109, psnr = 39.2549
Epoch 70: loss = 0.0100, psnr = 39.9700
Epoch 80: loss = 0.0093, psnr = 40.6273
Epoch 90: loss = 0.0087, psnr = 41.2550
Epoch 100: loss = 0.0081, psnr = 41.8756
Epoch 110: loss = 0.0075, psnr = 42.4985
Epoch 120: loss = 0.0070, psnr = 43.1278
Epoch 130: loss = 0.0065, psnr = 43.7724
Epoch 140: loss = 0.0060, psnr = 44.4449
Epoch 150: loss = 0.0055, psnr = 45.1598
Epoch 160: loss = 0.0050, psnr = 45.9399
Epoch 170: loss = 0.0046, psnr = 46.8027
Epoch 180: loss = 0.0041, psnr = 47.6987
Epoch 190: loss = 0.0037, psnr = 48.5600
Epoch 200: loss = 0.0034, psnr = 49.3554
Epoch 210: loss = 0.0031, psnr = 50.0752
Epoch 220: loss = 0.0029, psnr = 50.7239
Epoch 230: loss = 0.0028, psnr = 50.9246
Epoch 240: loss = 0.0026, ps

loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,▁▃▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇█████████████
loss,0.00099
mse,0.00099
psnr,60.12111


Epoch 0: loss = 0.3256, psnr = 9.7452
Epoch 10: loss = 0.0724, psnr = 22.8105
Epoch 20: loss = 0.0232, psnr = 32.7040
Epoch 30: loss = 0.0140, psnr = 37.0583
Epoch 40: loss = 0.0110, psnr = 39.1427
Epoch 50: loss = 0.0094, psnr = 40.5715
Epoch 60: loss = 0.0083, psnr = 41.6300
Epoch 70: loss = 0.0075, psnr = 42.5402
Epoch 80: loss = 0.0068, psnr = 43.3764
Epoch 90: loss = 0.0062, psnr = 44.1766
Epoch 100: loss = 0.0057, psnr = 44.9505
Epoch 110: loss = 0.0052, psnr = 45.7120
Epoch 120: loss = 0.0047, psnr = 46.4774
Epoch 130: loss = 0.0043, psnr = 47.2697
Epoch 140: loss = 0.0039, psnr = 48.1083
Epoch 150: loss = 0.0035, psnr = 49.0118
Epoch 160: loss = 0.0032, psnr = 49.9690
Epoch 170: loss = 0.0035, psnr = 49.2328
Epoch 180: loss = 0.0027, psnr = 51.5333
Epoch 190: loss = 0.0024, psnr = 52.5476
Epoch 200: loss = 0.0021, psnr = 53.4970
Epoch 210: loss = 0.0020, psnr = 54.1580
Epoch 220: loss = 0.0018, psnr = 54.8088
Epoch 230: loss = 0.0017, psnr = 55.3722
Epoch 240: loss = 0.0016, ps

loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
psnr,▁▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██▇███████████████
loss,0.00068
mse,0.00068
psnr,63.29368
